In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV


In [78]:
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import OneHotEncoder

# Updated dataset creation with 30 entries
data_places = {
    "Name": [
        "Goa", "Manali", "Rishikesh", "Agra", "Jaipur",
        "Shimla", "Darjeeling", "Alleppey", "Varanasi", "Amritsar",
        "Mysore", "Ooty", "Andaman and Nicobar Islands", "Lakshadweep",
        "Jaisalmer", "Leh", "Pondicherry", "Kanyakumari", "Shillong", "Coorg",
        "Mumbai", "Bengaluru", "Chennai", "Kolkata", "Hyderabad",
        "Nashik", "Ajmer", "Kodaikanal", "Mahabaleshwar", "Khajuraho",
        "Udaipur"
    ],
    "State": [
        "Goa", "Himachal Pradesh", "Uttarakhand", "Uttar Pradesh", "Rajasthan",
        "Himachal Pradesh", "West Bengal", "Kerala", "Uttar Pradesh", "Punjab",
        "Karnataka", "Tamil Nadu", "Andaman and Nicobar Islands", "Lakshadweep",
        "Rajasthan", "Jammu and Kashmir", "Puducherry", "Tamil Nadu", "Meghalaya", "Karnataka",
        "Maharashtra", "Karnataka", "Tamil Nadu", "West Bengal", "Telangana",
        "Maharashtra", "Rajasthan", "Tamil Nadu", "Maharashtra", "Madhya Pradesh",
        "Rajasthan"
    ],
    "Type": [
        "Beach", "Mountain", "Place of God", "Historical", "Historical",
        "Mountain", "Mountain", "Beach", "Religious", "Historical",
        "Cultural", "Mountain", "Beach", "Beach",
        "Desert", "Mountain", "Beach", "Place of God", "Mountain", "Mountain",
        "City Life", "City Life", "City Life", "City Life", "City Life",
        "Historical", "Religious", "Mountain", "Hill Station", "Historical",
        "Historical"
    ],
    "Climatic Condition": [
        "Tropical", "Cold", "Tropical", "Temperate", "Temperate",
        "Cold", "Cold", "Tropical", "Tropical", "Temperate",
        "Tropical", "Tropical", "Tropical", "Tropical",
        "Desert", "Cold", "Tropical", "Tropical", "Cold", "Tropical",
        "Tropical", "Tropical", "Tropical", "Tropical", "Tropical",
        "Tropical", "Tropical", "Cold", "Cold", "Temperate",
        "Temperate"
    ],
    "Budget": [
        "Medium", "High", "Low", "Medium", "Medium",
        "High", "Medium", "Medium", "Low", "Low",
        "Medium", "Medium", "High", "High",
        "Medium", "High", "Medium", "Low", "Low", "Medium",
        "Medium", "Medium", "Medium", "Medium", "Medium",
        "High", "Medium", "Medium", "Medium", "High",
        "Medium"
    ]
}

# Create DataFrame
df_places = pd.DataFrame(data_places)

# One-Hot Encoding for categorical features
encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(df_places[['Type', 'Climatic Condition', 'Budget']])

# Create a DataFrame for the encoded features
encoded_df = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out())

# Prepare feature matrix
X = encoded_df

# Create a k-NN model
knn = NearestNeighbors(n_neighbors=5, metric='euclidean')
knn.fit(X)

def recommend_places(user_preferences):
    # Encode user preferences
    user_input = pd.DataFrame({
        'Type': [user_preferences['type_of_place']],
        'Climatic Condition': [user_preferences['climatic_condition']],
        'Budget': [user_preferences['budget']]
    })

    user_encoded = encoder.transform(user_input)

    # Find nearest neighbors
    distances, indices = knn.kneighbors(user_encoded)

    # Get recommended places and their distances
    recommended_places = df_places.iloc[indices[0]].copy()  # Copy to avoid SettingWithCopyWarning
    distances = distances.flatten()  # Flatten distances for easier readability

    # Add distances to the DataFrame
    recommended_places['Distance'] = distances

    # Filter to ensure the recommendations match user preferences
    filtered_recommendations = recommended_places[
        (recommended_places['Type'] == user_preferences['type_of_place']) &
        (recommended_places['Climatic Condition'] == user_preferences['climatic_condition']) &
        (recommended_places['Budget'] == user_preferences['budget'])
    ]

    # If no recommendations, check for alternatives based on other preferences
    if filtered_recommendations.empty:
        message = "No recommendations found based on your exact preferences. Here are alternatives:"
        
        # Check for recommendations based on climatic condition
        climate_alternatives = recommended_places[
            (recommended_places['Climatic Condition'] == user_preferences['climatic_condition'])
        ]
        climate_message = "Alternatives based on climatic condition:\n" if not climate_alternatives.empty else "No alternatives found based on climatic condition.\n"

        # Check for recommendations based on budget
        budget_alternatives = recommended_places[
            (recommended_places['Budget'] == user_preferences['budget'])
        ]
        budget_message = "Alternatives based on budget:\n" if not budget_alternatives.empty else "No alternatives found based on budget.\n"

        # Output alternatives
        return message, climate_message, climate_alternatives[['Name', 'State', 'Type', 'Climatic Condition', 'Budget']], budget_message, budget_alternatives[['Name', 'State', 'Type', 'Climatic Condition', 'Budget']]

    return filtered_recommendations[['Name', 'State', 'Type', 'Climatic Condition', 'Budget']]

# Example user input
user_input = {
    'type_of_place': 'Mountain',
    'climatic_condition': 'Desert',
    'budget': 'High'
}

recommended_places = recommend_places(user_input)

# Print the recommendations or message
if isinstance(recommended_places, tuple):
    print(recommended_places[0])  # Print the main message
    print(recommended_places[1])  # Print climate alternatives message
    print(recommended_places[2])  # Print climate alternatives
    print(recommended_places[3])  # Print budget alternatives message
    print(recommended_places[4])  # Print budget alternatives
else:
    print(recommended_places)



No recommendations found based on your exact preferences. Here are alternatives:
No alternatives found based on climatic condition.

Empty DataFrame
Columns: [Name, State, Type, Climatic Condition, Budget]
Index: []
Alternatives based on budget:

      Name              State      Type Climatic Condition Budget
5   Shimla   Himachal Pradesh  Mountain               Cold   High
1   Manali   Himachal Pradesh  Mountain               Cold   High
15     Leh  Jammu and Kashmir  Mountain               Cold   High


C:\Users\abeer\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
